# Flowmeter data processing

Takes data from flowmeter saved with Teraterm (eg. od a line "2018-10-21, 03:56:44, +000.48 +023.28 +008.19") and converts it to an excel document vith columns: 'Time, min','Flow, mL/min','Pressure, bar', 'Temp., °C', 'Flux, L/m3h', and 'Flux_cor_temp, L/m3h'.

Enter the filename:

In [9]:
fileName = 'test_data'

Define how long the time intervale in s (not smaller than 1 s) between mesurements should be:

In [10]:
intervalS = 5

Memb

In [11]:
area = 0.046

Data processing

In [17]:
import pandas as pd
import re
import numpy as np
import seaborn as sb

file = fileName + '.txt'  
data = []
errors =[]

with open(file) as f:
    read_data = f.read()
    data = read_data.split('\n')
    clean_data = []
    for line in data:
        m_data = re.findall(r".\d\d\d\.\d\d", line, flags=0)
        timestamp = re.findall(r"\d\d\d\d-\d\d-\d\d, \d\d\:\d\d\:\d\d", line, flags=0)
        #eg. "2018-10-21, 03:58:20"
        if len(m_data) == 3 and len(timestamp) == 1:
            press,temp,flux = m_data
            time = timestamp[0]
            try:
                clean_data.append([pd.Timestamp(time), float(flux), float(press)*0.0689475729, float(temp)])
            except:
                errors.append(timestamp + m_data)
                pass


df = pd.DataFrame(clean_data, columns=['Time, min','Flow, mL/min','Pressure, bar', 'Temp., °C'])
df['Time, min'] = df['Time, min'] - df['Time, min'][0]
df['Time, min'] = df['Time, min'].dt.total_seconds()/60
df['Flux, L/m3h']= df['Flow, mL/min']*0.001*60/area
temp = df['Temp., °C']

def temp_conv(temp):
    #corects the flux for viscosity with an empirical function
    return (1.79934+temp*(-0.04316)+temp*temp*0.000376037)

df['Flux_cor_temp, L/m3h'] = df['Flux, L/m3h']*temp_conv(temp)
df = df.groupby(['Time, min'],as_index=False).mean().dropna(how='all')
#df = df.groupby(np.arange(len(df))//intervalS).mean()
 
writer = pd.ExcelWriter(fileName + '.xlsx')
df.to_excel(writer, 'Sheet1')
writer.save()
df.head()

,"Time, min","Flow, mL/min","Pressure, bar","Temp., °C","Flux, L/m3h","Flux_cor_temp, L/m3h"
0,0.000000,9.29,0.033784,21.93,12.117391,12.525587
1,0.083333,9.29,0.033784,21.94,12.117391,12.522356
2,0.166667,9.29,0.033784,21.94,12.117391,12.522356
3,0.250000,9.31,0.033784,21.94,12.143478,12.549315
4,0.350000,9.31,0.033784,21.94,12.143478,12.549315
